# Ayudantía 08: Networking - pickle

## Ejemplo AC09 2019-1: Ejemplo "Doggo" (Modificado) [Solución]

### Autores: [@bmurtagh01 ](https://github.com/bmurtagh01) & [@StroveLight ](https://github.com/StroveLight ) & [@JoacoCoyu](https://github.com/JoacoCoyu)

## Enunciado

Deberás de establecer una conexión entre un servidor y un cliente mediante
el uso de sockets, para crear un sistema de transferencia de archivos. Para lograr esto, el cliente deberá poder
realizar solicitudes a través de comandos que el servidor deberá responder.

El objetivo de tu programa es:
* poder ver los archivos ubicados en el servidor
* poder recibir la imagen ubicada en el servidor
* implementar un protocolo de recepcion de datos en el que los primeros bytes indican el largo del contenido

Recuerda que el numero de puerto debe ser mayor a 1023, porque los numeros inferiores son para elementos del sistema operativo del computador. Te recomendamos intentar un numero de puerto superior a 9000 para no chocar con otros programas

**Notar que el código se presentará de forma fraccionada (dividido en varias celdas) para facilitar la lectura, pero puedes ver y probar los archivos de solución en la carpeta** `Ejemplo pickle AC09 - 2019-1 (Doggo)/`

## Servidor (archivo server.py)

### Función Servidor.receive()
Originalmente, en el esqueleto entregado se poseía esta función:

In [ ]:
def receive(self):
    """
    [MODIFICAR]
    Recibe datos binarios del cliente, a través del socket,
    cumpliendo con el protocolo establecido en el enunciado.
    """
    return self.cliente.recv(128)  # maldición, esto es demasiado poco.

...pero esa funcion no recibía todos los datos (solo 128 bytes).

Por lo tanto se deberá implementar un sistema que indique cuantos datos se van a enviar.

Usualmente se envían primero 4 bytes que indican el tamaño del archivo que se va a enviar, para luego enviar el archivo o mensaje completo; para seguir este formato será necesario modificar el send() del cliente y del servidor posteriormente, pero por ahora asumamos que se está siguiendo este sistema.

In [ ]:
def receive(self):
    # primero se reciven los primeros 4 bytes que indican el largo del mensaje
    numero_de_bytes_en_bytes = self.cliente.recv(4)

    # luego se convierte el numero de bytes que se van a recibir a int
    numero_de_bytes_a_recibir = int.from_bytes(numero_de_bytes_en_bytes, byteorder='big')

    # posteriormente se crea un bytearray para recibir el mensaje
    mensaje = bytearray()

    # y luego se agrega el mensaje por trozos de 2048 bytes hasta que se complete
    while len(mensaje) > numero_de_bytes_a_recibir:
        # se recibe aquí un trozo de 2048 bytes y se agrega a lo que se lleva del mensaje
        mensaje.extend(self.socket.recv(2048))
    return mensaje

El método anterior parece hacer todo de buena manera, sin embargo posee un problema:

**Se pueden recibir mas de la cantidad de bytes del mensaje**

Pues si el mensaje tiene, por ejemplo, tamaño 5.100 bytes, se recibirán correctamente dos bloques de 2048 bytes, pero el tercer trozo tendrá solamente 1004 bytes, mientras que el socket está intentado recibir 2048.

El hacer esto puede causar que se pierda información de mensajes posteriores, pues esta será considerada como parte del primer mensaje (si es que ya están en la cola), además de poder causar un error con el JsonEncoder (al esperar un valor distinto de bytes del que está)

Debido a lo anterior necesitamos hacer unos ajustes para que se reciba el mensaje por partes, pero sin recibir de mas.

In [ ]:
def receive(self):
    # (...) el mismo codigo de arriba hasta el while (...)
    numero_de_bytes_en_bytes = self.cliente.recv(4)
    numero_de_bytes_a_recibir = int.from_bytes(numero_de_bytes_en_bytes, byteorder='big')
    mensaje = bytearray()
    
    # primero, se indica que la cantidad de numero de bytes a recibir será mayor a cero
    # esto debido a que se usará la variable como numero de bytes restantes
    while numero_de_bytes_a_recibir > 0:
        
        # Luego, si es que se necesitan recibir mas de 2048 bytes
        # se recibirán solo 2048, en caso contrario,
        # se recibiran solo los que quedan
        largo_por_recibir = min(numero_de_bytes_a_recibir, 2048)
        
        # se recibe aquí un trozo de 2048 bytes y se agrega a lo que se lleva del mensaje
        mensage.extend(self.cliente.recv(largo_por_recibir))
        
        # hay que recordar actualizar la variable despues
        numero_de_bytes -= largo_por_recibir
        
    return mensaje

### Función Server.send()
Originalmente, en el esqueleto entregado se poseía esta función:

In [ ]:
def send(self, mensaje):
        """
        [MODIFICAR]
        Envía datos binarios al servidor conectado por el socket,
        cumpliendo con el protocolo establecido en el enunciado.
        """
        self.socket.sendall(mensaje)

...pero ahora debemos de implementar el protocolo mencionado anteriormente, enviar primero 4 bytes indicando el largo y luego el resto del mensaje.

La funcion quedaría así:

In [ ]:
def send(self, mensaje):
    
    # Primero se convierte el largo del mensaje a 4 bytes
    largo_mensaje_en_bytes = int.to_bytes(len(mensaje), byteorder='big', length=4)
    
    # Luego se envían esos 4 bytes
    self.cliente.sendall(largo_mensaje_en_bytes)
    
    # Finalmente se envía el mensaje
    self.cliente.sendall(mensaje)

### Función Server.lista_directorio()
Originalmente se tenía lo siguiente:

In [ ]:
def lista_directorio(self):
    """
    [COMPLETAR]
    Envía al cliente una lista que contiene los nombres de
    todos los archivos existentes en la carpeta del servidor.
    """
    pass

Ahora, usando la librería ```os``` obtendremos el directorio actual, como una lista, y luego podemos seriailzar la información, usando ```pickle.dumps()```, para que pueda ser enviada.

In [ ]:
def lista_directorio(self):

    # Se serializa la informacion
    message = pickle.dumps(os.listdir('.'))

    # Y luego se usa el método send ya implementado para enviar
    self.send(message)

### Función Server.enviar_archivo()
Para esta función se espera poder enviar todo el contenido de un archivo desde el servidor al cliente.

Originalmente se teníamos lo siguiente:

In [ ]:
def enviar_archivo(self, ruta_archivo):
    """
    [COMPLETAR]
    Envía al cliente un archivo ubicado en el directorio del servidor.
    """
    pass

Como solo es posible mandar bytes a través de los sockets, podemos abrir el archivo con formato de lectura de bytes, de esta forma, no es necesario serializarlo, pues ya tenemos los bytes que lo representan.

In [ ]:
def enviar_archivo(self, ruta_archivo):
    
    # Se abre el archivo en forma de lectura de bytes
    with open(ruta_archivo, 'rb') as file:
        
        # Se guarda toda la informacion del archivo en la variable data
        data = file.read()
        
    # Se usa el metodo send() previamente implementado para enviar la informacion
    self.send(data)

## Cliente (archivo cliente.py)

### Función Cliente.recibir()
Para esta función se hace algo muy similar a la función recibir del servidor (solo cambian los nombres de las variables), resultando en la siguiente funcion:

In [ ]:
def receive(self):
    # obtenemos el largo del mensaje en 4 bytes y lo transformamos a entero
    numero_de_bytes = int.from_bytes(self.socket.recv(4), byteorder='big')
    # creamos el arreglo que contendra al mensaje
    message = bytearray()
    
    # recibimios el resto del mensaje
    while numero_de_bytes > 0:
        largo_por_recibir = min(numero_de_bytes, 2048)
        message.extend(self.socket.recv(largo_por_recibir))
        numero_de_bytes -= largo_por_recibir

    return message

### Función Cliente.send()
Al igual que la funcion anterior, hay pocos cambios con respecto a su equivalente en el servidor. Esta funcion quedaría así:

In [ ]:
def send(self, mensaje):
    # Obtenemos, convertimos y enviamos el largo en 4 bytes
    self.socket.sendall(int.to_bytes(len(mensaje), byteorder='big', length=4))
    # Enviamos el resto del mensaje
    self.socket.sendall(mensaje)

### Funcion Cliente.ls()
Esta funcion se encargará de mostrar el directorio en pantalla.

Para esta función no había esqueleto y su version completa es la siguiente:

In [ ]:
def ls(self):

    # primero se usa el metodo ya implementado para recibir la lista de archivos
    data = self.receive()

    # luego se deserializa la informacion recibida
    data2 = pickle.loads(data)

    # Y finalmente se imprime cada elemento de la lista obtenida
    for archivo in data2:
        print(' -', archivo)

### Función Cliente.descargar()

Recordando como armamos el envío del archivo, sabemos que para esta función ya se están recibiendo los datos en forma de bytes y sin serializar, por lo que no necesitamos deserializarlos. La función resultante será:

In [ ]:
def descargar(self, ruta_archivo):
    # se recibe la informacion
    data = self.receive()
    
    # Abrimos el archivo en forma de bytes, pero esta vez en formato de escritura
    with open(ruta_archivo, 'wb') as file:
        # escribimos la infromación recibida
        file.write(data)
    print('Archivo creado')

Teniendo ya todas las funciones del cliente y del servidor, basta con hacer la conexión de ambos en los main de cada archivo, por lo que te invitamos a leer, ejecutar y si te animas modificar los archivos entregados como solución de este problema.

Cabe señalar que la anterior es solamente una posible solución del ejercicio, pero existen múltiples maneras de hacer un programa que funcione correctamente y cumpla con lo pedido, por lo que si tu solución es distinta, pero funciona sin problemas, probablemente también esté correcta :D

Un ejemplo de una solución alternativa, es haber realizado la descarga de archivos serializando su contenido usando pickle antes de ser enviado; lo que estará correcto siempre y cuando se revierta el proceso al momento de recibir el archivo.